In [21]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Define the input tensors for stocks and sales
n = 1000
torch.manual_seed(0)
stocks = torch.randint(0, 10, (n,))
demand = torch.poisson(torch.ones(n) * 2.0)
sales = torch.min(demand, stocks)



In [10]:
# Define the loss function
def poisson_loss(predictions, sales, stocks):
    poisson_loss = torch.mean(predictions - sales * torch.log(predictions + 1e-10))  # Adding a small epsilon to avoid log(0)
    exceeded_stock_penalty = torch.mean(torch.max(predictions - stocks, torch.zeros_like(predictions)))
    loss = poisson_loss + exceeded_stock_penalty
    return loss

def mse_loss_function(predictions,sales,stocks):
    mse_loss = nn.functional.mse_loss(predictions, sales)
    exceeded_stock_penalty = torch.mean(torch.max(predictions - stocks, torch.zeros_like(predictions)))
    loss = mse_loss + exceeded_stock_penalty
    return loss


def forecast_loss(predictions, sales, stocks, loss_function="poisson"):
    if loss_function =="poisson":
        return poisson_loss(predictions,sales,stocks)
    elif loss_function=="mse":
        return mse_loss_function(predictions,sales,stocks)



In [14]:
class MeanModel(nn.Module):
    def __init__(self):
        super(MeanModel, self).__init__()
        self.mean = nn.Parameter(torch.randn(1))

    def forward(self, n):
        return self.mean * torch.ones((n,), dtype=torch.float32)
        


In [12]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True

In [23]:
# Convert data to DataLoader for convenience
train_dataset = TensorDataset(stocks, demand, sales)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model and optimize using gradient descent with weight decay and early stopping
model = MeanModel()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)

# Define early stopping
early_stopping = EarlyStopping(tolerance=5, min_delta=10)
num_epochs  = 100
losses = list()
# Training loop
for epoch in range(100):
    model.train()
    for batch_stocks, batch_demand, batch_sales in train_loader:
        optimizer.zero_grad()
        outputs = model(len(batch_stocks))
        loss = forecast_loss(outputs, batch_sales, batch_stocks, "poisson")
        loss.backward()
        optimizer.step()
        losses.append(loss)
    # Validation loop (assuming you have a validation dataset)
    model.eval()
    # validation_loss = compute_validation_loss(model, validation_data)  # Replace with your validation loss computation
    # Uncomment the line above when you have a validation dataset

    # Check for early stopping
    # early_stopping(validation_loss, model)
    # Uncomment the line above when you have a validation dataset

    print("Epoch:", epoch, "Loss:", loss.item())
  

Epoch: 0 Loss: nan
Epoch: 1 Loss: nan
Epoch: 2 Loss: nan
Epoch: 3 Loss: nan
Epoch: 4 Loss: nan
Epoch: 5 Loss: nan
Epoch: 6 Loss: nan
Epoch: 7 Loss: nan
Epoch: 8 Loss: nan
Epoch: 9 Loss: nan
Epoch: 10 Loss: nan
Epoch: 11 Loss: nan
Epoch: 12 Loss: nan
Epoch: 13 Loss: nan
Epoch: 14 Loss: nan
Epoch: 15 Loss: nan
Epoch: 16 Loss: nan
Epoch: 17 Loss: nan
Epoch: 18 Loss: nan
Epoch: 19 Loss: nan
Epoch: 20 Loss: nan
Epoch: 21 Loss: nan
Epoch: 22 Loss: nan
Epoch: 23 Loss: nan
Epoch: 24 Loss: nan
Epoch: 25 Loss: nan
Epoch: 26 Loss: nan
Epoch: 27 Loss: nan
Epoch: 28 Loss: nan
Epoch: 29 Loss: nan
Epoch: 30 Loss: nan
Epoch: 31 Loss: nan
Epoch: 32 Loss: nan
Epoch: 33 Loss: nan
Epoch: 34 Loss: nan
Epoch: 35 Loss: nan
Epoch: 36 Loss: nan
Epoch: 37 Loss: nan
Epoch: 38 Loss: nan
Epoch: 39 Loss: nan
Epoch: 40 Loss: nan
Epoch: 41 Loss: nan
Epoch: 42 Loss: nan
Epoch: 43 Loss: nan
Epoch: 44 Loss: nan
Epoch: 45 Loss: nan
Epoch: 46 Loss: nan
Epoch: 47 Loss: nan
Epoch: 48 Loss: nan
Epoch: 49 Loss: nan
Epoch: 50 

In [24]:

# Plotting the loss
plt.plot(range(1, num_epochs + 1), losses, marker='o', linestyle='-', color='b')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time, loss: poisson')
plt.grid(True)
plt.show()

NameError: name 'losses' is not defined